## Connect to SQL Server database

In [2]:
import pyodbc
import pandas as pd

In [4]:
# Check the driver for your SQL Server version - 
pyodbc.drivers()

In [10]:
# Create a connection string for SQL Server - replace values with yours
connection_string = ("Driver={YOUR SQL SERVER DRIVER NAME};"
	"Server=YOUR SERVER NAME;"
	"Database=YOUR DATABASE NAME;"
	"UID=YOUR USER NAME;"
	"PWD=YOUR PASSWORD")

In [12]:
# Create variable 'conn' that connects to the database using the connection string
conn = pyodbc.connect(connection_string)

In [14]:
# Create a variable 'cursor' created from calling the cursor method on the conn variable
cursor = conn.cursor()

## Create Product and Category tables

In [37]:
# Create a products and a category table - execute the query via the cursor variable
cursor.execute('''
CREATE TABLE category(
category_id INT PRIMARY KEY IDENTITY (1,1),
category_name VARCHAR(100) NOT NULL
);

CREATE TABLE product(
product_id INT PRIMARY KEY IDENTITY (1,1),
product_name VARCHAR(100) NOT NULL,
short_description VARCHAR(150) NOT NULL,
price DECIMAL(10,2) NOT NULL,
category_id INT NOT NULL,
FOREIGN KEY (category_id) REFERENCES category(category_id)
);

''')

## Insert Data into Tables

In [39]:
# Insert some categories and products
cursor.execute('''
INSERT INTO category(category_name)
VALUES
('Health and Fitness'),
('Technology'),
('Pets'),
('Home furnishings');

INSERT INTO product(product_name, short_description, price, category_id)
VALUES
('Running machine','Excellent running machine', 299.99,1),
('Exercise bike','Ride at home', 199.99,1),
('Dog bed','Your pet will sleep well', 49.99,3),
('Laptop','Powerful 8gb RAM laptop', 349.99,2),
('4 seater sofa','Comfortable and comes in 3 styles', 549.99,4);

''')


## Run SQL queries

In [42]:
# Run a simple query to check that the products have been added
cursor.execute('''
SELECT * FROM product
''')

for row in cursor:
    print(row)
    

(1, 'Running machine', 'Excellent running machine', Decimal('299.99'), 1)
(2, 'Exercise bike', 'Ride at home', Decimal('199.99'), 1)
(3, 'Dog bed', 'Your pet will sleep well', Decimal('49.99'), 3)
(4, 'Laptop', 'Powerful 8gb RAM laptop', Decimal('349.99'), 2)
(5, '4 seater sofa', 'Comfortable and comes in 3 styles', Decimal('549.99'), 4)


In [44]:
# Run a join query to get the products with their category name
cursor.execute('''
SELECT product.product_name, product.short_description, product.price, category.category_name
FROM product
INNER JOIN category
ON product.category_id = category.category_id
''')

## Convert SQL inner join query result to Pandas DataFrame

In [43]:
# Create an empty list variable
# This will be a list of lists that we can use to create a pandas data frame
data_list = list()

In [45]:
# Append every row returned from the query into a list.
for row in cursor:
    # Convert each row to a list
    row_as_list = list(row)
    # Append (add) each row to the data list (list made up of lists)
    data_list.append(row_as_list)

# Check the data list looks ok
data_list

[['Running machine',
  'Excellent running machine',
  Decimal('299.99'),
  'Health and Fitness'],
 ['Exercise bike', 'Ride at home', Decimal('199.99'), 'Health and Fitness'],
 ['Dog bed', 'Your pet will sleep well', Decimal('49.99'), 'Pets'],
 ['Laptop', 'Powerful 8gb RAM laptop', Decimal('349.99'), 'Technology'],
 ['4 seater sofa',
  'Comfortable and comes in 3 styles',
  Decimal('549.99'),
  'Home furnishings']]

In [47]:
# Create a pandas data frame 
# NOTE: The column names MUST match the column names from the data list
df = pd.DataFrame(data_list, columns=['product_name', 'short_description', 'price', 'category_name'])

In [48]:
# Check data frame is as expected
df

,product_name,short_description,price,category_name
0,Running machine,Excellent running machine,299.99,Health and Fitness
1,Exercise bike,Ride at home,199.99,Health and Fitness
2,Dog bed,Your pet will sleep well,49.99,Pets
3,Laptop,Powerful 8gb RAM laptop,349.99,Technology
4,4 seater sofa,Comfortable and comes in 3 styles,549.99,Home furnishings
